1. Install Required Libraries

In [1]:
!pip install PyMuPDF
!pip install sentence-transformers
!pip install faiss-cpu
!pip install transformers
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 30.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1

2. Extract Text from PDFs


In [2]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
import fitz  # PyMuPDF

def extract_text_from_pdfs(pdf_paths):
    texts = []
    for pdf_path in pdf_paths:
        doc = fitz.open(pdf_path)
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            texts.append(page.get_text())
    return texts

pdf_paths = ['debra_satz_why_some_things_should_not_be_for_sale.pdf', 'The_Luxury_Strategy_Break_the_Rules_of_Marketing_to_Build_Luxury_Brands.pdf']  # Replace with your PDF paths
pdf_texts = extract_text_from_pdfs(pdf_paths)

sentence_model = SentenceTransformer('all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
import numpy as np

# Initialize sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Create embeddings for the extracted texts
embeddings = model.encode(pdf_texts, convert_to_tensor=False)

# Convert embeddings to numpy array
embeddings = np.array(embeddings)

In [6]:
import faiss
# Create a FAISS index
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

In [11]:
index.ntotal

602

In [7]:
len(embeddings)

602

In [39]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM

# Initialize the language model
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

# Create a pipeline
nlp = pipeline('summarization', model=model, tokenizer=tokenizer, device=device)

def retrieve_and_answer(question, index, texts, model, nlp_pipeline, top_k=5):
    # Encode the question
    question_embedding = sentence_model.encode([question], convert_to_tensor=False)

    # Retrieve top-k relevant texts
    D, I = index.search(np.array(question_embedding), top_k)

    # Concatenate retrieved texts
    retrieved_texts = " ".join([texts[i] for i in I[0]])

    # Generate answer using the language model (since bart have the max input text length of 1024)
    if len(retrieved_texts) > 1024:
      answer = nlp_pipeline(retrieved_texts[:1024], max_length=200, min_length=30, do_sample=False)
    else:
      answer = nlp_pipeline(retrieved_texts, max_length=200, min_length=30, do_sample=False)

    return answer[0]['summary_text']

In [25]:
question = "Can you tell me about the non-return effect?"
question_embedding = sentence_model.encode([question], convert_to_tensor=False)
question_embedding

array([[-4.81736399e-02,  3.16461269e-03,  2.50174254e-02,
         1.09902896e-01,  4.89618368e-02, -1.05892997e-02,
        -1.58053003e-02, -1.06474562e-02,  7.56986514e-02,
        -5.76517079e-03,  1.20854720e-01,  3.00807953e-02,
        -1.23465089e-02, -8.87382627e-02, -4.93286103e-02,
        -5.85077479e-02,  1.11505650e-01,  1.89227089e-02,
        -9.97143313e-02,  1.69504173e-02,  4.19155061e-02,
        -1.04958430e-01,  1.54405516e-02,  7.75290653e-03,
        -1.19604832e-02, -4.75246795e-02, -6.71981201e-02,
         4.59559401e-03,  2.59426180e-02, -1.26850978e-01,
        -1.96604449e-02,  1.08277939e-01, -6.28668219e-02,
        -5.85171953e-03, -7.34952465e-02,  8.92986879e-02,
        -8.93113110e-03,  8.47629271e-03, -2.03134064e-02,
         3.19451690e-02, -5.00444323e-02,  2.14398298e-02,
         9.63421352e-03,  2.47093644e-02,  7.41621330e-02,
        -4.93534841e-02,  3.57701117e-03, -5.43634854e-02,
        -4.34791371e-02, -5.30940108e-02,  2.14760359e-0

In [26]:
D, I = index.search(np.array(question_embedding), 5)
D, I

(array([[1.0299428, 1.4269722, 1.4470589, 1.4477854, 1.468566 ]],
       dtype=float32),
 array([[293, 456, 225, 119, 206]]))

In [27]:
pdf_texts[293]

'The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nincrease in spending power, globalization and communications – when they\nare all working at full throttle they help propel luxury to previously\nunknown levels.\nBefore moving on to other things, we need to add a word about a well-\nknown effect of luxury, namely the ‘non-return effect’, or ‘ratchet effect’, the\ncounterpart of the ‘non-linear increase’ that we talked about earlier: once\npeople have tasted luxury in whatever area, it is very difficult for them to turn\naway from it – to come back to earth. When there is a fall in spending power,\nwe witness a symmetrical phenomenon to the one that we identified when\nspending power was increased: people start cutting their expenditure on\nevery kind of conventional product (for example, they will trade their\nChevrolet for a smaller and more economic Ford), but it’s a different matter\nwhen it comes to luxury products (they will keep 

In [28]:
# Concatenate retrieved texts
retrieved_texts = " ".join([pdf_texts[i] for i in I[0]])
retrieved_texts

'The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nincrease in spending power, globalization and communications – when they\nare all working at full throttle they help propel luxury to previously\nunknown levels.\nBefore moving on to other things, we need to add a word about a well-\nknown effect of luxury, namely the ‘non-return effect’, or ‘ratchet effect’, the\ncounterpart of the ‘non-linear increase’ that we talked about earlier: once\npeople have tasted luxury in whatever area, it is very difficult for them to turn\naway from it – to come back to earth. When there is a fall in spending power,\nwe witness a symmetrical phenomenon to the one that we identified when\nspending power was increased: people start cutting their expenditure on\nevery kind of conventional product (for example, they will trade their\nChevrolet for a smaller and more economic Ford), but it’s a different matter\nwhen it comes to luxury products (they will keep 

In [35]:
retrieved_texts[:1024]

'The non-return effect\nSo, we have identified four powerful drivers at work here: democratization,\nincrease in spending power, globalization and communications – when they\nare all working at full throttle they help propel luxury to previously\nunknown levels.\nBefore moving on to other things, we need to add a word about a well-\nknown effect of luxury, namely the ‘non-return effect’, or ‘ratchet effect’, the\ncounterpart of the ‘non-linear increase’ that we talked about earlier: once\npeople have tasted luxury in whatever area, it is very difficult for them to turn\naway from it – to come back to earth. When there is a fall in spending power,\nwe witness a symmetrical phenomenon to the one that we identified when\nspending power was increased: people start cutting their expenditure on\nevery kind of conventional product (for example, they will trade their\nChevrolet for a smaller and more economic Ford), but it’s a different matter\nwhen it comes to luxury products (they will keep 

In [36]:
# Create a pipeline
nlp = pipeline('summarization', model=model, tokenizer=tokenizer, device=device)

# Generate answer using the language model
answer = nlp(retrieved_texts[:1024], max_length=200, min_length=30, do_sample=False)
answer

[{'summary_text': 'The ‘non-return effect’ is the counterpart of the ‘ non-linear increase’ that we talked about earlier. Once people have tasted luxury, it is very difficult for them to turn away from it. When there is a fall in spending power, we witness a symmetrical phenomenon.'}]

In [38]:
question = "Can you tell me about the non-return effect?"
answer = retrieve_and_answer(question, index, pdf_texts, model, nlp)
print("Answer:", answer)

Answer: The ‘non-return effect’ is the counterpart of the ‘ non-linear increase’ that we talked about earlier. Once people have tasted luxury, it is very difficult for them to turn away from it. When there is a fall in spending power, we witness a symmetrical phenomenon.
